У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/customer_segmentation_train.csv')

In [4]:
# Перегляд перших рядків датасету
print(train_df.head())

# Перевірка розміру датасету
print(train_df.shape)

# Перевірка інформації про датасет
print(train_df.info())

# Перевірка наявності пропущених значень
print(train_df.isnull().sum())

       ID  Gender Ever_Married  Age Graduated     Profession  Work_Experience  \
0  462809    Male           No   22        No     Healthcare              1.0   
1  462643  Female          Yes   38       Yes       Engineer              NaN   
2  466315  Female          Yes   67       Yes       Engineer              1.0   
3  461735    Male          Yes   67       Yes         Lawyer              0.0   
4  462669  Female          Yes   40       Yes  Entertainment              NaN   

  Spending_Score  Family_Size  Var_1 Segmentation  
0            Low          4.0  Cat_4            D  
1        Average          3.0  Cat_4            A  
2            Low          1.0  Cat_6            B  
3           High          2.0  Cat_6            B  
4           High          6.0  Cat_6            A  
(8068, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           -------------- 

In [6]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Визначення числових і категоріальних колонок
numerical_cols = train_df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = train_df.select_dtypes(include=['object']).columns

# Видалення цільової змінної з переліку колонок
categorical_cols = categorical_cols.drop('Segmentation')

# Побудова пайплайну для числових даних
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

# Побудова пайплайну для категоріальних даних
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Об'єднання трансформерів у єдиний колонковий трансформер
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Поділ на тренувальну та тестувальну вибірки
X = train_df.drop('Segmentation', axis=1)
y = train_df['Segmentation']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Застосування пайплайну до тренувальних даних
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Перевірка результатів трансформації
print(X_train.shape)
print(X_test.shape)

(6454, 29)
(1614, 29)


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках і порівняти лін регресію на апсампл даних без категоріальних ознак.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [9]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Виділення числових ознак
X_train_numeric = X_train[:, :len(numerical_cols)]
X_test_numeric = X_test[:, :len(numerical_cols)]

# Ресемплінг з базовим SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_numeric, y_train)

# Тренування моделі
model = LogisticRegression()
model.fit(X_train_smote, y_train_smote)

# Оцінка на тестових даних
y_pred = model.predict(X_test_numeric)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           A       0.34      0.24      0.28       391
           B       0.33      0.07      0.12       369
           C       0.39      0.52      0.44       380
           D       0.49      0.77      0.60       474

    accuracy                           0.42      1614
   macro avg       0.39      0.40      0.36      1614
weighted avg       0.39      0.42      0.38      1614



In [12]:
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek

# Визначення індексів категоріальних ознак
categorical_feature_indices = [i for i in range(len(numerical_cols), X_train.shape[1])]

# SMOTENC
smotenc = SMOTENC(categorical_features=categorical_feature_indices, random_state=42)
X_train_smotenc, y_train_smotenc = smotenc.fit_resample(X_train, y_train)

# SMOTE-Tomek
smote_tomek = SMOTETomek(smote=smotenc, random_state=42)
X_train_smote_tomek, y_train_smote_tomek = smote_tomek.fit_resample(X_train, y_train)

# Тренування моделі на даних з SMOTENC
model_smotenc = LogisticRegression()
model_smotenc.fit(X_train_smotenc, y_train_smotenc)

# Тренування моделі на даних з SMOTE-Tomek
model_smote_tomek = LogisticRegression()
model_smote_tomek.fit(X_train_smote_tomek, y_train_smote_tomek)

# Оцінка на тестових даних
y_pred_smotenc = model_smotenc.predict(X_test)
print("SMOTENC:")
print(classification_report(y_test, y_pred_smotenc))

y_pred_smote_tomek = model_smote_tomek.predict(X_test)
print("SMOTE-Tomek:")
print(classification_report(y_test, y_pred_smote_tomek))

SMOTENC:
              precision    recall  f1-score   support

           A       0.34      0.24      0.28       391
           B       0.33      0.07      0.12       369
           C       0.39      0.53      0.45       380
           D       0.49      0.77      0.60       474

    accuracy                           0.43      1614
   macro avg       0.39      0.40      0.36      1614
weighted avg       0.39      0.43      0.38      1614

SMOTE-Tomek:
              precision    recall  f1-score   support

           A       0.00      0.00      0.00       391
           B       0.00      0.00      0.00       369
           C       0.24      1.00      0.38       380
           D       0.00      0.00      0.00       474

    accuracy                           0.24      1614
   macro avg       0.06      0.25      0.10      1614
weighted avg       0.06      0.24      0.09      1614



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [14]:
#модель на оригінальних даних

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

# Навчання моделі на оригінальних даних
model_original = OneVsRestClassifier(LogisticRegression())
model_original.fit(X_train, y_train)

# Оцінка на тестових даних
y_pred_original = model_original.predict(X_test)
print("Original Data:")
print(classification_report(y_test, y_pred_original))

Original Data:
              precision    recall  f1-score   support

           A       0.00      0.00      0.00       391
           B       0.00      0.00      0.00       369
           C       0.37      0.41      0.39       380
           D       0.35      0.89      0.50       474

    accuracy                           0.36      1614
   macro avg       0.18      0.32      0.22      1614
weighted avg       0.19      0.36      0.24      1614



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
# Навчання моделі на даних з SMOTE
model_smote = OneVsRestClassifier(LogisticRegression())
model_smote.fit(X_train_smote, y_train_smote)

# Оцінка на тестових даних
y_pred_smote = model_smote.predict(X_test_numeric)
print("SMOTE:")
print(classification_report(y_test, y_pred_smote))

SMOTE:
              precision    recall  f1-score   support

           A       0.00      0.00      0.00       391
           B       0.00      0.00      0.00       369
           C       0.24      1.00      0.38       380
           D       0.00      0.00      0.00       474

    accuracy                           0.24      1614
   macro avg       0.06      0.25      0.10      1614
weighted avg       0.06      0.24      0.09      1614



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
# Навчання моделі на даних з SMOTE-Tomek
model_smote_tomek = OneVsRestClassifier(LogisticRegression())
model_smote_tomek.fit(X_train_smote_tomek, y_train_smote_tomek)

# Оцінка на тестових даних
y_pred_smote_tomek = model_smote_tomek.predict(X_test)
print("SMOTE-Tomek:")
print(classification_report(y_test, y_pred_smote_tomek))

SMOTE-Tomek:
              precision    recall  f1-score   support

           A       0.00      0.00      0.00       391
           B       0.00      0.00      0.00       369
           C       0.33      0.77      0.46       380
           D       0.50      0.77      0.60       474

    accuracy                           0.41      1614
   macro avg       0.21      0.38      0.27      1614
weighted avg       0.22      0.41      0.29      1614



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Висновки
**Original Data**

Модель на оригінальних даних показує невисоку точність для класів A і B (точність 0.00), але кращу точність для класів C і D.
Загальна точність (accuracy) складає 0.36.
Модель добре розпізнає клас D (recall 0.89), але значення f1-score для інших класів невисокі.

**SMOTE**

Модель з SMOTE значно покращує recall для класу C до 1.00, але це призводить до низької точності для інших класів.
Загальна точність (accuracy) знизилася до 0.24.
Ця модель не змогла добре розпізнати класи A, B і D (точність і recall 0.00).

**SMOTE-Tomek**

Модель з SMOTE-Tomek показала кращі результати, ніж SMOTE, особливо для класів C і D.
Загальна точність (accuracy) складає 0.41, що є найвищим серед усіх моделей.
Значення f1-score для класу D становить 0.60, а для класу C – 0.46.
Вибір метрики
Для порівняння моделей ми можемо використовувати f1-score, оскільки він враховує як точність (precision), так і повноту (recall), і підходить для нерівномірно розподілених даних.

**Отже**, модель, збалансована з SMOTE-Tomek, показала найкращі результати, оскільки вона має найвищу загальну точність (accuracy) 0.41.
Значення f1-score для класів C і D є вищими порівняно з іншими моделями.
Вона демонструє збалансованіші показники для різних класів.

**Гіпотеза**
Незважаючи на те, що SMOTE і SMOTE-Tomek зазвичай використовуються для покращення балансу в даних, у цьому випадку SMOTE знизив точність моделей для класів A і B. Це може бути пов'язано з тим, що моделі не отримали достатньої кількості якісних зразків для цих класів. SMOTE-Tomek, з іншого боку, показав кращі результати, оскільки він не тільки додає нові зразки, але й видаляє шумові зразки, що допомагає покращити загальну точність моделі.